In [ ]:
import pandas as pd
import numpy as np
from math import radians, sin, cos, sqrt, atan2

# read in the data
data = pd.read_csv('data.csv')
#==========================================================================================================================
def point_position(p3, p4, new_point, test_point):
    # Calculate the equation of the line in the form y = mx + c
    m = (p4[0] - p3[0]) / (p4[1] - p3[1])
    c = p3[0] - m * p3[1]

    # Calculate the equation of the perpendicular line in the form y = mx + c
    mp = -1 / m  # negative reciprocal slope
    cp = new_point[0] - mp * new_point[1]

    # Calculate the intersection point of the two lines
    xi = (cp - c) / (m - mp)
    yi = m * xi + c

    # Check whether the test point is on the left or right side of the perpendicular line
    test_vector = np.array(test_point) - np.array(new_point)
    perp_vector = np.array([1, mp])
    dot_product = np.dot(test_vector, perp_vector)

    if dot_product > 0:
        return 0
    elif dot_product < 0:
        return 1
    else:
        return 0

    
#==========================================================================================================================

def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # radius of the earth in km

    # convert degrees to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])

    # calculate haversine
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    distance = R * c

    return distance*1000
#==========================================================================================================================

class Point:
    def __init__(self, x, y):
    self.x = x
    self.y = y

class line:
    def __init__(self, p1, p2):
    self.p1 = p1
    self.p2 = p2

def onLine(l1, p):
    # Check whether p is on the line or not
    if (
        p.x <= max(l1.p1.x, l1.p2.x)
        and p.x <= min(l1.p1.x, l1.p2.x)
        and (p.y <= max(l1.p1.y, l1.p2.y) and p.y <= min(l1.p1.y, l1.p2.y))
    ):
        return True
    return False

def direction(a, b, c):
    val = (b.y - a.y) * (c.x - b.x) - (b.x - a.x) * (c.y - b.y)
    if val == 0:
    # Collinear
        return 0
    elif val < 0:
        # Anti-clockwise direction
        return 2
    # Clockwise direction
    return 1

def isIntersect(l1, l2):
    # Four direction for two lines and points of other line
    dir1 = direction(l1.p1, l1.p2, l2.p1)
    dir2 = direction(l1.p1, l1.p2, l2.p2)
    dir3 = direction(l2.p1, l2.p2, l1.p1)
    dir4 = direction(l2.p1, l2.p2, l1.p2)

    # When intersecting
    if dir1 != dir2 and dir3 != dir4:
        return True

    # When p2 of line2 are on the line1
    if dir1 == 0 and onLine(l1, l2.p1):
        return True

    # When p1 of line2 are on the line1
    if dir2 == 0 and onLine(l1, l2.p2):
        return True

    # When p2 of line1 are on the line2
    if dir3 == 0 and onLine(l2, l1.p1):
        return True

    # When p1 of line1 are on the line2
    if dir4 == 0 and onLine(l2, l1.p2):
        return True

    return False

def checkInside(poly, n, p):
    # When polygon has less than 3 edge, it is not polygon
    if n < 3:
        return False

    # Create a point at infinity, y is same as point p
    exline = line(p, Point(9999, p.y))
    count = 0
    i = 0
    while True:
        # Forming a line from two consecutive points of poly
        side = line(poly[i], poly[(i + 1) % n])
        if isIntersect(side, exline):
            # If side is intersects ex
            if (direction(side.p1, p, side.p2) == 0):
                return onLine(side, p);
            count += 1

        i = (i + 1) % n;
        if i == 0:
            break

    # When count is odd
    return count & 1;

#==========================================================================================================================
polygon1 = [ Point(30.898458, 75.872615), Point(30.898847, 75.871570), Point(30.898744, 75.871486), Point(30.898256, 75.872419) ];
polygon2 = [ Point(30.898744, 75.871486), Point(30.898256, 75.872419), Point(30.897982, 75.872304), Point(30.898610, 75.871468) ];

# Initialize an empty list to store the results
results = []

# Group the data by frames
frames = data.groupby('frame')

p3 = [30.898744, 75.871486]
p4 = [30.898256, 75.872419]

# Loop over each frame
for name, group in frames:
    # Filter pedestrians and vehicles
    pedestrians = group[group['Class'] == ' Pedestrian']
    vehicles = group[group['Class'].isin([' Bicycle',' Tuk-Tuk',' Car',' Motorcycle',' Light Truck',' Tractor',' Bus',' Van'])]
    
    # Loop over each pedestrian
    for _, ped_row in pedestrians.iterrows():
        # Calculate pedestrian coordinates
        ped_x, ped_y = ped_row['X'], ped_row['Y']
        p1 = Point(ped_x, ped_y)
        new_point = [ped_x, ped_y]
        # Initialize minimum distance and closest vehicle ID
        min_dist = float('inf')
        closest_vehicle_id = None
        closest_vehicle_class = 'No_vehicle'
        closest_vehicle_speed = 0
        count = 0
        pedestrian_speed = ped_row['Speed']
        # Loop over each vehicle
        for _, veh_row in vehicles.iterrows():
            # Calculate vehicle coordinates
            veh_x, veh_y = veh_row['X'], veh_row['Y']
            v1 = Point(veh_x, veh_y)
            test_point = [veh_x, veh_y]
            
            # Check if vehicle and pedestrian are in the same rectangle
            if ((checkInside(polygon1, 4, p1) and checkInside(polygon1, 4, v1) and (point_position(p3, p4, new_point, test_point))) or 
                (checkInside(polygon2, 4, p1) and checkInside(polygon2, 4, v1) and not(point_position(p3, p4, new_point, test_point)))):
                # Calculate distance between pedestrian and vehicle
                dist = haversine(ped_x, ped_y, veh_x, veh_y)
                count = count + 1
                # Check if distance is smaller than minimum distance so far
                if dist < min_dist:
                    min_dist = dist
                    closest_vehicle_id = veh_row['ID']
                    closest_vehicle_class = veh_row['Class']
                    closest_vehicle_speed = veh_row['Speed']
        # Append a new row to the results list with the relevant information
        results.append({'frame': name, 'ID': ped_row['ID'],'pedestrian_speed': pedestrian_speed, 'closest_vehicle_id': closest_vehicle_id, 'closest_vehicle_dist': min_dist, 'closest_vehicle_class': closest_vehicle_class,'closest_vehicle_speed': closest_vehicle_speed, 'approching_vehicle_count': count})

# Convert the list of dictionaries into a DataFrame
closest_vehicle = pd.DataFrame(results)